In [3]:
import numpy as np
from scipy.integrate import solve_ivp
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import os
from joblib import Parallel, delayed
from datetime import datetime

# Конфигурация генерации
manual_params = {
    'beta': 19.6783, 'Tm': 0.2995, 'Te': 0.0256, 'Tfc': 0.1038,
    'kfc': 1.6613, 'b': 0.7839, 'h0': 0.7304, 'x0': 1.0180, 'Tp': 0.7938,
}

config = {
    'N': 1000,
    'T': [0, 20, 200], # начало, остановка, количество точек
    'param_ranges': {
        'Kp': [0.0, 3.0],
        'Ki': [0.0, 2.0],
    },
    'setpoint_range': [50, 600],
    'noise_level': 0.02,
    'noise_enabled': False,
    'n_jobs': -1,
    'reg_lambda': 0.5,
    'os_coeffs': {'Kp_ratio': 0.2, 'Ki_ratio': 0.15}  # коэффициенты обратной связи
}

# Логирование
log_dir = "generation_logs"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f"generation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")

def log_message(message):
    with open(log_file, "a") as f:
        f.write(f"{datetime.now().isoformat()} [INFO] {message}\n")
    print(message)

# Сохранение конфигурации для воспроизводимости
with open(os.path.join(log_dir, "config.json"), "w") as f:
    json.dump(config, f, indent=2)

# Основные параметры симуляции
T = np.linspace(config['T'][0], config['T'][1], config['T'][2])
dt = T[1] - T[0]
n_states = 5
X0 = np.zeros(n_states)

# Модель объекта
def plant_update(t, x, u, params):
    beta, Tm, Te, Tfc, kfc, b, h0, x0, Tp = params
    x1, x2, x3, x4, x5 = x
    dx1_dt = (1.0 / (beta * Tm)) * (x2 - x1)
    dx2_dt = (beta / Te) * x3 - (beta / Te) * x1 - (1.0 / Te) * x2
    dx3_dt = (kfc / Tfc) * x4 - (1.0 / Tfc) * x3
    k_leak = 5.0
    dx5_dt = ((b / Tp) + ((2.0 * h0 * x0) / Tp)) * x1 - (1.0 / Tp) * x5 - k_leak * x5
    dx4_dt = u[0]
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt]

# ПИ-регулятор
class PIControllerWithOS:
    def __init__(self, Kp, Ki, dt, Kp_os=0.0, Ki_os=0.0, u_min=0, u_max=3000, integral_min=-20, integral_max=20):
        self.Kp = Kp
        self.Ki = Ki
        self.Kp_os = Kp_os
        self.Ki_os = Ki_os
        self.dt = dt
        self.integral = 0.0
        self.u_min = u_min
        self.u_max = u_max
        self.integral_min = integral_min
        self.integral_max = integral_max
        self.x1_prev = None
        self.t_prev = None

    def reset(self):
        self.integral = 0.0
        self.x1_prev = None
        self.t_prev = None

    def __call__(self, setpoint, measurement, x1, t):
        if config.get('noise_enabled', False):
            noise = config['noise_level'] * setpoint * np.random.randn()
            measurement += noise
        error = setpoint - measurement
        if self.t_prev is None:
            dt_local = 0.0
        else:
            dt_local = t - self.t_prev
        self.t_prev = t
        self.integral += error * dt_local
        self.integral = np.clip(self.integral, self.integral_min, self.integral_max)
        dx1_dt_num = (x1 - self.x1_prev) / dt_local if (self.x1_prev is not None and dt_local > 0) else 0.0
        self.x1_prev = x1
        u = (self.Kp * error + self.Ki * self.integral - self.Kp_os * dx1_dt_num - self.Ki_os * x1)
        return np.clip(u, self.u_min, self.u_max)

# Генерация одного сэмпла
def generate_sample(seed):
    np.random.seed(seed)
    try:
        # Параметры объекта
        params_obj = [manual_params[k] for k in ['beta','Tm','Te','Tfc','kfc','b','h0','x0','Tp']]

        # Случайные параметры регулятора
        Kp = np.random.uniform(*config['param_ranges']['Kp'])
        Ki = np.random.uniform(*config['param_ranges']['Ki'])
        
        # Расчет параметров обратной связи
        Kp_os = Kp * config['os_coeffs']['Kp_ratio']
        Ki_os = Ki * config['os_coeffs']['Ki_ratio']

        setpoint = np.random.uniform(*config['setpoint_range'])
        
        pi = PIControllerWithOS(Kp, Ki, dt, Kp_os, Ki_os)
        pi.reset()

        def closed_loop_rhs(t, x):
            measurement = x[4]
            x1 = x[0]
            u = pi(setpoint, measurement, x1, t)
            dx = plant_update(t, x, [u], params_obj)
            if np.any(np.isnan(x)) or np.any(np.abs(x) > 1e6):
                return np.zeros_like(x)
            return dx

        sol = solve_ivp(closed_loop_rhs, [T[0], T[-1]], X0, t_eval=T, method='RK45', rtol=1e-6, atol=1e-8)
        if not sol.success:
            return None

        p_hist = sol.y[4]
        pressure_hist = p_hist * 0.00980665
        pressure_setpoint = setpoint * 0.00980665
        IAE_pressure = np.sum(np.abs(pressure_hist - pressure_setpoint)) * dt
        RMSE_pressure = np.sqrt(np.mean((pressure_hist - pressure_setpoint) ** 2))
        overshoot = (np.max(pressure_hist) - pressure_setpoint) / pressure_setpoint * 100

        # Регуляризация параметров
        reg_lambda = config.get('reg_lambda', 0.5)
        reg_penalty = reg_lambda * (Kp**2 + Ki**2)
        IAE_pressure_reg = IAE_pressure + reg_penalty

        # Критерий качества (1 - хороший, 0 - плохой)
        is_good = int((RMSE_pressure < 0.03 * setpoint) and (abs(overshoot) <= 20))

        return (
            [setpoint, config['noise_level'], IAE_pressure_reg, RMSE_pressure, overshoot, is_good],
            [Kp, Ki, Kp_os, Ki_os] + params_obj
        )

    except Exception as e:
        log_message(f"Ошибка генерации сэмпла: {str(e)}")
        return None

# Параллельная генерация данных
if __name__ == "__main__":
    log_message("Запуск генерации данных...")

    results = Parallel(n_jobs=config['n_jobs'])(
        delayed(generate_sample)(seed)
        for seed in tqdm(range(config['N']), desc="Генерация данных")
    )

    # Обработка результатов
    X_train, y_train = [], []
    bad_samples = 0

    for result in results:
        if result is None:
            bad_samples += 1
        else:
            X_train.append(result[0])
            y_train.append(result[1])

    log_message(f"Успешно сгенерировано: {len(X_train)}/{config['N']}")
    log_message(f"Отброшено плохих сэмплов: {bad_samples}")

    # Сохранение данных
    X_train = np.array(X_train, dtype=np.float32)
    y_train = np.array(y_train, dtype=np.float32)
    np.savez('train_data_simulated_checked.npz', X_train=X_train, y_train=y_train)
    log_message("Данные сохранены в train_data_simulated_checked.npz")

    # Визуализация распределения метрик
    plt.figure(figsize=(10, 6))
    plt.hist([x[2] for x in X_train], bins=50, alpha=0.5, label='IAE+reg')
    plt.hist([x[3] for x in X_train], bins=50, alpha=0.5, label='RMSE')
    plt.hist([x[4] for x in X_train], bins=50, alpha=0.5, label='Overshoot')
    plt.xlabel('Значение метрики')
    plt.ylabel('Частота')
    plt.title('Распределение метрик качества')
    plt.legend()
    plt.savefig(os.path.join(log_dir, 'metrics_distribution.png'))
    plt.close()
    log_message("Визуализация сохранена в metrics_distribution.png")

    # Вывод лучших примеров
    best_examples = []
    for x, y in zip(X_train, y_train):
        setpoint = x[0]
        RMSE = x[3]
        overshoot = x[4]
        if RMSE < 0.03 * setpoint and abs(overshoot) <= 20:
            best_examples.append((x, y))
    best_examples = sorted(best_examples, key=lambda tup: tup[0][2])
    print('\nЛучшие примеры:')
    for i, (x, y) in enumerate(best_examples[:5]):
        print(f"Пример {i+1}:")
        print(f"  setpoint={x[0]:.2f}, IAE+reg={x[2]:.4f}, RMSE={x[3]:.4f}, overshoot={x[4]:.2f}%")
        print(f"  Kp={y[0]:.3f}, Ki={y[1]:.3f}")
        print(f"  Kp_os={y[-2]:.3f}, Ki_os={y[-1]:.3f}\n")


Запуск генерации данных...


Генерация данных:   3%|▎         | 32/1000 [00:18<10:21,  1.56it/s]

KeyboardInterrupt: 